##### 21BDS0340 - Abhinav Dinesh Srivatsa

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns

In [ ]:
data = np.load("ORL_faces.npz")
X_train = data["trainX"]
X_train_norm = np.array(X_train, dtype="float") / 255
X_test = data["testX"]
X_test_norm = np.array(X_test, dtype="float") / 255
y_train = data["trainY"]
y_test = data["testY"]
X_train_norm.shape, y_train.shape

In [ ]:
rows, cols = 112, 92
im_shape = (-1, rows, cols)

X_train_reshaped = np.reshape(X_train_norm, im_shape)
X_test_reshaped = np.reshape(X_test_norm, im_shape)
X_train_reshaped.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_reshaped, y_train, test_size=0.2, random_state=42, shuffle=True)
X_train.shape, X_val.shape

In [ ]:
def random_face(data):
    n = int(random.random() * len(data))
    plt.imshow(data[n])

In [ ]:
random_face(X_train)

In [ ]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Input((rows, cols, 1)),
    tf.keras.layers.Conv2D(36, 7, activation="relu"),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Conv2D(54, 5, activation="relu"),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(20, activation="softmax")
])

model1.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history1 = model1.fit(
    X_train, y_train,
    epochs=100, validation_data=(X_val, y_val)
)

In [ ]:
X_test_reshaped.shape

In [ ]:
model1.evaluate(X_test_reshaped, y_test, verbose=0)

In [ ]:
plt.plot(history1.history["accuracy"])
plt.plot(history1.history["val_accuracy"])
plt.title("Model Accuracies")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["train", "val"], loc="upper left")

In [ ]:
plt.plot(history1.history["loss"])
plt.plot(history1.history["val_loss"])
plt.title("Model Losses")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["train", "val"], loc="upper left")

In [ ]:
def confusion_matrix(model, test_features, test_labels):
    y_pred = model.predict(test_features)
    y_pred = [np.argmax(row) for row in y_pred]
    matrix = np.zeros((20, 20))
    for i in range(len(y_pred)):
        real = test_labels[i]
        pred = y_pred[i]
        matrix[real][pred] += 1
    plt.figure(figsize=(10, 7))
    sns.heatmap(matrix, cmap="mako", annot=True)
    plt.title("Model Confusion Matrix")
    plt.xlabel("Real")
    plt.ylabel("Predicted")
    return y_pred

In [ ]:
y_pred = confusion_matrix(model1, X_test_reshaped, y_test)

In [ ]:
report = classification_report(y_test, y_pred)
print(report)